In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing, tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
# If you are using Google Colab, upload the proper files from Kaggle. 
# If you are using Google Colab, you can use ls to help you figure out what directory you are in.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train[train['Survived']== 1].groupby('Age').size().sort_values()

Age
0.42      1
55.00     1
53.00     1
47.00     1
43.00     1
         ..
22.00    11
35.00    11
27.00    11
36.00    11
24.00    15
Length: 65, dtype: int64

In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test.groupby('Embarked').size()
test[test['Ticket']== '1601']

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S


In [20]:
X_train = pd.get_dummies(train[['Pclass', 'Sex', 'Embarked']])
X_train['SibSp'] = train['SibSp']
X_train['Parch'] = train['Parch']
#train_people = train['Parch'] + train['SibSp']
#X_train['People'] = (train_people - np.mean(train_people)) / np.std(train_people)
X_train['Age'] = train['Age'].replace(np.NaN, np.mean(train['Age']))
#X_train['binary_Age'] = 0
#X_train['binary_Age'] =  np.where((X_train['Age'] >= 35), 1,
                               #X_train['binary_Age'])

#X_train['log_age'] = np.log(X_train['log_age'])

#X_train['normalize_age'] = (X_train['Age'] - np.mean(X_train['Age'])) / np.std(X_train['Age'])
#X_train['normalize_Fare'] = (train['Fare'] - np.mean(train['Fare'])) / np.std(train['Fare'])

X_train['Fare'] = train['Fare']
X_train['Fare'] = np.where((X_train['Pclass'] == 3) & (X_train['Fare'] == 0), 
                               np.mean(train[train['Pclass'] == 3]['Fare']), 
                               X_train['Fare'])
X_train['Fare'] = np.where((X_train['Pclass'] == 2) & (X_train['Fare'] == 0), 
                               np.mean(train[train['Pclass'] == 2]['Fare']), 
                               X_train['Fare'])
X_train['Fare'] = np.where((X_train['Pclass'] == 1) & (X_train['Fare'] == 0), 
                               np.mean(train[train['Pclass'] == 1]['Fare']), 
                               X_train['Fare'])
#X_train['square_Fare'] = np.square(X_train['Fare'])
X_train['normalize_Fare'] = (X_train['Fare'] - np.mean(X_train['Fare'])) / np.std(X_train['Fare'])


X_train['Cabin'] = train['Cabin']
X_train['Cabin'] = X_train['Cabin'].str[0]
X_train['Cabin'] = X_train['Cabin'].replace({'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 2, 'F': 2, 'G': 2, 'T': 2})
#X_train['Cabin'] = np.where(X_train['Cabin'].notna(), 1, X_train['Cabin'])
X_train['Cabin'] = np.where(X_train['Cabin'].isna(), 0, X_train['Cabin'])

X_train['P+S'] = X_train['SibSp'] + X_train['Parch']
#X_train['square_Pclass'] = np.square(X_train['Pclass'])

X_train = X_train.drop(columns = ['Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Cabin', 'Parch', 'SibSp'])

In [21]:
X_train

,Pclass,Sex_female,Sex_male,Age,normalize_Fare,P+S
0,3,0,1,22.000000,-0.516026,1
1,1,1,0,38.000000,0.773351,1
2,3,1,0,26.000000,-0.502434,0
3,1,1,0,35.000000,0.407211,1
4,3,0,1,35.000000,-0.499917,0
...,...,...,...,...,...,...
886,2,0,1,27.000000,-0.400244,0
887,1,1,0,19.000000,-0.057931,0
888,3,1,0,29.699118,-0.189822,3
889,1,0,1,26.000000,-0.057931,0


In [22]:
Y_train = train[['Survived']]
model = RandomForestClassifier(n_estimators = 3000, random_state = 43)
# Train the model on training data
model.fit(X_train, Y_train)
y_pred = model.predict(X_train)
accuracy_score(Y_train, y_pred)
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
#print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

<ipython-input-22-3736b865f780>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, Y_train)


0.9820426487093153

In [23]:
#from sklearn.ensemble import ExtraTreesClassifier
#model = ExtraTreesClassifier(n_estimators=3000, max_depth=None,
                             #min_samples_split=2, random_state=43)
#model.fit(X_train, Y_train)
#y_pred = model.predict(X_train)
#accuracy_score(Y_train, y_pred)

In [24]:
from sklearn.model_selection import KFold
from sklearn.base import clone

def cross_validate_rmse(model):
    model = clone(model)
    five_fold = KFold(n_splits=5)
    accuracy = []
    for tr_ind, va_ind in five_fold.split(train):
        model.fit(X_train.iloc[tr_ind,:], Y_train['Survived'].iloc[tr_ind])
        #model.fit(X_train, Y_train)
        #tr['mpg'].iloc[va_ind], model.predict(tr.iloc[va_ind,:])
        accuracy.append(accuracy_score(Y_train['Survived'].iloc[va_ind], model.predict(X_train.iloc[va_ind,:])))
        print(accuracy)
    return np.mean(accuracy)

In [25]:
cross_validate_rmse(model)

[0.7653631284916201]
[0.7653631284916201, 0.8370786516853933]
[0.7653631284916201, 0.8370786516853933, 0.8426966292134831]
[0.7653631284916201, 0.8370786516853933, 0.8426966292134831, 0.7865168539325843]
[0.7653631284916201, 0.8370786516853933, 0.8426966292134831, 0.7865168539325843, 0.8595505617977528]


0.8182411650241669

In [26]:
#Y_train = train['Survived']
#clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=100, 
                    #activation = 'logistic', solver='lbfgs', verbose=1,  
                    #random_state=43)

#clf.fit(X_train,Y_train)

#clf = tree.DecisionTreeClassifier(max_depth=5)
#clf = clf.fit(X_train, Y_train)

#y_pred = clf.predict(X_train)

#accuracy_score(Y_train, y_pred)

#print('Accuracy on training---')
#y_pred_train=clf.predict(X_train)
#print(accuracy_score(Y_train,y_pred_train))

In [27]:
X_test = pd.get_dummies(test[['Pclass', 'Sex', 'Embarked']])
X_test['SibSp'] = test['SibSp']
X_test['Parch'] = test['Parch']
#test_people = test['Parch'] + test['SibSp']
#X_test['People'] = (test_people - np.mean(test_people)) / np.std(test_people)
X_test['Age'] = test['Age'].replace(np.NaN, np.mean(test['Age']))
#X_test['normalize_age'] = (X_test['normalize_age'] - np.mean(X_test['normalize_age'])) / np.std(X_test['normalize_age'])
#X_test['normalize_Fare'] = test['Fare'].replace(np.NaN, np.mean(test['Fare']))
#X_test['normalize_Fare'] = (X_test['normalize_Fare'] - np.mean(X_test['normalize_Fare'])) / np.std(X_test['normalize_Fare'])

X_test['Fare'] = test['Fare'].replace(np.NaN, 0)
X_test['Fare'] = np.where((X_test['Pclass'] == 3) & (X_test['Fare'] == 0 ), 
                               np.mean(test[test['Pclass'] == 3]['Fare']), 
                               X_test['Fare'])
X_test['Fare'] = np.where((X_test['Pclass'] == 2) & (X_test['Fare'] == 0), 
                               np.mean(test[test['Pclass'] == 2]['Fare']), 
                               X_test['Fare'])
X_test['Fare'] = np.where((X_test['Pclass'] == 1) & (X_test['Fare'] == 0), 
                               np.mean(test[test['Pclass'] == 1]['Fare']), 
                               X_test['Fare'])
X_test['normalize_Fare'] = (X_test['Fare'] - np.mean(X_test['Fare'])) / np.std(X_test['Fare'])
X_test['P+S'] = X_test['SibSp'] + X_test['Parch']
#X_test['Cabin'] = test['Cabin']
#X_test['Cabin'] = X_test['Cabin'].str[0]
#X_test['Cabin'] = X_test['Cabin'].replace({'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 2, 'F': 2, 'G': 2, 'T': 2})
#X_test['Cabin'] = np.where(X_test['Cabin'].notna(), 1, X_test['Cabin'])
#X_test['Cabin'] = np.where(X_test['Cabin'].isna(), 0, X_test['Cabin'])
X_test = X_test.drop(columns = ['Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Parch', 'SibSp'])
X_test

,Pclass,Sex_female,Sex_male,Age,normalize_Fare,P+S
0,3,0,1,34.50000,-0.504568,0
1,3,1,0,47.00000,-0.519408,1
2,2,0,1,62.00000,-0.471311,0
3,3,0,1,27.00000,-0.489655,0
4,3,1,0,22.00000,-0.424780,2
...,...,...,...,...,...,...
413,3,0,1,30.27259,-0.500617,0
414,1,1,0,39.00000,1.304247,0
415,3,0,1,38.50000,-0.514934,0
416,3,0,1,30.27259,-0.500617,0


In [28]:
y_pred_test=model.predict(X_test)

In [29]:
#accuracy_score(test['Survived'], y_pred_test)

In [30]:
test['Survived'] = y_pred_test

In [31]:
final_prediction = test[['PassengerId', 'Survived']]
final_prediction.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [32]:
final_prediction.to_csv('my_prediction.csv', index = False)